Here we have tried modelling a CNN on a dataset of drug-side effect pair. But we only consider side effect groups which is 23 labels. And this is a multi-label classification problem where dataset is highly skewed.

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np

In [2]:
df_data = pd.read_csv('dataframes/SIDER_with_isoSMILES.tsv', sep='\t')
df_data

,Unnamed: 0,STITCH,SMILES,Blood and lymphatic system disorders,Cardiac disorders,"Congenital, familial and genetic disorders",Ear and labyrinth disorders,Endocrine disorders,Eye disorders,Gastrointestinal disorders,...,Musculoskeletal and connective tissue disorders,"Neoplasms benign, malignant and unspecified (incl cysts and polyps)",Nervous system disorders,"Pregnancy, puerperium and perinatal conditions",Psychiatric disorders,Renal and urinary disorders,Reproductive system and breast disorders,"Respiratory, thoracic and mediastinal disorders",Skin and subcutaneous tissue disorders,Vascular disorders
0,2,CID000000143,Nc1nc(=O)c2c([nH]1)NCC(CNc1ccc(C(=O)NC(CCC(=O)O)C(=O)O)cc1)N2C=O,1,0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,25,CID000000772,CC(=O)NC1C(O)OC(COS(=O)(=O)O)C(OC2OC(C(=O)O)C(OC3OC(CO)C(OC4OC(C(=O)O)C(O)C(O)C4OS(=O)(=O)O)C(OS(=O)(=O)O)C3NS(=O)(=O)O)C(O)C2OS(=O)(=O)O)C1O,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,50,CID000002019,Cc1c2oc3c(C)ccc(C(=O)NC4C(=O)NC(C(C)C)C(=O)N5CCCC5C(=O)N(C)CC(=O)N(C)C(C(C)C)C(=O)OC4C)c3nc-2c(C(=O)NC2C(=O)NC(C(C)C)C(=O)N3CCCC3C(=O)N(C)CC(=O)N(C)C(C(C)C)C(=O)OC2C)c(N)c1=O,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,66,CID000002156,CCCCc1oc2ccccc2c1C(=O)c1cc(I)c(OCC[NH+](CC)CC)c(I)c1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,70,CID000002162,CCOC(=O)C1=C(COCCN)NC(C)=C(C(=O)OC)C1c1ccccc1Cl,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,1549,CID070683024,COCCOC(=O)NCC(=O)N[C@H]1[C@H]([C@H](O)[C@H](O)CO)O[C@@](OC[C@H]2O[C@H](O[C@H](C)[C@H](N)C(=O)O)[C@H](NC(C)=O)[C@@H](O)[C@H]2O)(C(=O)O)C[C@@H]1O,1,1,1,1,0,1,1,...,1,0,1,1,0,0,0,1,1,1
821,1550,CID070685014,C[C@@H]1NC(=O)[C@@H]2CCCN2C(=O)[C@H](CC(N)=O)NC(=O)[C@@H]2CSSC[C@@H](N)C(=O)N[C@H]3CSSC[C@H](NC1=O)C(=O)N[C@@H]([C@@H](C)O)C(=O)NCC(=O)N[C@H](C(=O)N[C@@H](Cc1ccc(O)cc1)C(=O)O)CSSC[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)[C@H](CCC(=O)O)NC3=O)C(=O)N2,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
822,1552,CID070788982,NCCCC[C@@H]1NC(=O)[C@@H](Cc2c[nH]c3ccccc23)NC(=O)[C@H](c2ccccc2)NC(=O)[C@@H]2C[C@@H](OC(=O)NCCN)CN2C(=O)[C@H](Cc2ccccc2)NC(=O)[C@H](Cc2ccc(OCc3ccccc3)cc2)NC1=O.N[C@@H](CC(=O)O)C(=O)O.N[C@@H](CC(=O)O)C(=O)O,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
823,1553,CID071306410,CC(C)(C)NCC(O)COc1nsnc1N1CCOCC1.CCNC1CN(CCCOC)S(=O)(=O)c2sc(S(N)(=O)=O)cc21,1,1,1,1,1,1,1,...,1,0,1,1,1,1,0,1,1,1


In [322]:
# random_sample_df = df_data.sample(n=400, random_state=42) # for pilot dataset
random_sample_df = df_data.sample(n=100) # for pilot dataset
# random_sample_df

In [344]:
# Data
# X = random_sample_df["SMILES"].values
# y = random_sample_df.iloc[:, 3:26].values
X = df_data["SMILES"].values
y = df_data.iloc[:, 3:26].values
# X = df_data["SMILES"].values
# y = df_data.iloc[:, 4:27].values
print(y.shape)
print(X.shape)
# print(y[:,1].shape)

(825, 23)
(825,)


In [345]:
max_seq_len = max(1, max(len(sequence) for sequence in X))
print(max_seq_len)

398


In [346]:
##-- Calculate class occurance in the sampled data --##
# class_vectors = np.array(y)
# class_count = np.sum(class_vectors, axis=0)
# print(len(class_count),  class_count, np.sum(class_count))


# Tokenizer

Convert our text input data into token indices. This means that every token (we can decide what a token is char, word, sub-word, etc.) is mapped to a unique index which allows us to represent our text as an array of indices.

In [5]:
import json
from collections import Counter
from more_itertools import take

In [6]:
class Tokenizer(object):
    def __init__(self, char_level, num_tokens=None,
                 pad_token="<PAD>", oov_token="<UNK>",
                 token_to_index=None):
        self.char_level = char_level
        self.separator = "" if self.char_level else " "
        if num_tokens: num_tokens -= 2 # pad + unk tokens
        self.num_tokens = num_tokens
        self.pad_token = pad_token
        self.oov_token = oov_token
        if not token_to_index:
            token_to_index = {pad_token: 0, oov_token: 1}
        self.token_to_index = token_to_index
        self.index_to_token = {v: k for k, v in self.token_to_index.items()}

    def __len__(self):
        return len(self.token_to_index)

    def __str__(self):
        return f"<Tokenizer(num_tokens={len(self)})>"

    def fit_on_texts(self, texts):
        if not self.char_level:
            texts = [text.split(" ") for text in texts]
        all_tokens = [token for text in texts for token in text]
        counts = Counter(all_tokens).most_common(self.num_tokens)
        self.min_token_freq = counts[-1][1]
        for token, count in counts:
            index = len(self)
            self.token_to_index[token] = index
            self.index_to_token[index] = token
        return self

    def texts_to_sequences(self, texts):
        sequences = []
        for text in texts:
            if not self.char_level:
                text = text.split(" ")
            sequence = []
            for token in text:
                sequence.append(self.token_to_index.get(
                    token, self.token_to_index[self.oov_token]))
            sequences.append(np.asarray(sequence))
        return sequences

    def sequences_to_texts(self, sequences):
        texts = []
        for sequence in sequences:
            text = []
            for index in sequence:
                text.append(self.index_to_token.get(index, self.oov_token))
            texts.append(self.separator.join([token for token in text]))
        return texts

    def save(self, fp):
        with open(fp, "w") as fp:
            contents = {
                "char_level": self.char_level,
                "oov_token": self.oov_token,
                "token_to_index": self.token_to_index
            }
            json.dump(contents, fp, indent=4, sort_keys=False)

    @classmethod
    def load(cls, fp):
        with open(fp, "r") as fp:
            kwargs = json.load(fp=fp)
        return cls(**kwargs)

In [347]:
# -- test --#
smiles = ['CN1C=NC2=C1C(=O)N(C(=O)N2C)C']
tokenizer = Tokenizer(char_level=True, num_tokens=62)
tokenizer.fit_on_texts(texts=smiles)
VOCAB_SIZE = len(tokenizer)
print (tokenizer)
print (take(5, tokenizer.token_to_index.items()))
print (f"least freq token's freq: {tokenizer.min_token_freq}") 
test = tokenizer.texts_to_sequences(smiles)
print(test)

<Tokenizer(num_tokens=10)>
[('<PAD>', 0), ('<UNK>', 1), ('C', 2), ('N', 3), ('=', 4)]
least freq token's freq: 2
[array([2, 3, 7, 2, 4, 3, 2, 8, 4, 2, 7, 2, 5, 4, 9, 6, 3, 5, 2, 5, 4, 9,
       6, 3, 8, 2, 6, 2])]


In [348]:
tokenizer = Tokenizer(char_level=True, num_tokens=62)
# tokenizer.fit_on_texts(texts=X_train)
tokenizer.fit_on_texts(texts=X)
VOCAB_SIZE = len(tokenizer)
print (tokenizer)
print (take(5, tokenizer.token_to_index.items()))
print (f"least freq token's freq: {tokenizer.min_token_freq}") 

<Tokenizer(num_tokens=47)>
[('<PAD>', 0), ('<UNK>', 1), ('C', 2), ('(', 3), (')', 4)]
least freq token's freq: 1


In [349]:
# Convert texts to sequences of indices
X = tokenizer.texts_to_sequences(X)
preprocessed_text = tokenizer.sequences_to_texts([X[0]])[0]
print ("Text to indices:\n"
    f"  (preprocessed) → {preprocessed_text}\n"
    f"  (tokenized) → {X[0]}")

Text to indices:
  (preprocessed) → Nc1nc(=O)c2c([nH]1)NCC(CNc1ccc(C(=O)NC(CCC(=O)O)C(=O)O)cc1)N2C=O
  (tokenized) → [13  5 12 16  5  3 10  6  4  5 14  5  3  8 16 11  9 12  4 13  2  2  3  2
 13  5 12  5  5  5  3  2  3 10  6  4 13  2  3  2  2  2  3 10  6  4  6  4
  2  3 10  6  4  6  4  5  5 12  4 13 14  2 10  6]


# One-hot encoding

OHE of the tokens will create matrix with binary values where each vocabulary or tokens will be indicated by 0 or 1.

In [10]:
def to_categorical(seq, num_classes):
    """One-hot encode a sequence of tokens."""
    one_hot = np.zeros((len(seq), num_classes)) # num_classes is the vocabulary size
    for i, item in enumerate(seq):
        one_hot[i, item] = 1.
    return one_hot

In [350]:
# Convert tokens to one-hot
vocab_size = len(tokenizer)
X = [to_categorical(seq, num_classes=vocab_size) for seq in X]
# X_train = [to_categorical(seq, num_classes=vocab_size) for seq in X_train]
# X_val = [to_categorical(seq, num_classes=vocab_size) for seq in X_val]
# X_test = [to_categorical(seq, num_classes=vocab_size) for seq in X_test]

In [416]:
print(len(X[0]), print(vocab_size))

47
64 None


# Padding
Our inputs are all of varying length but we need each batch to be uniformly shaped. Therefore, we will use padding to make all the inputs in the batch the same length.

Here we will create a batch of shape (N (i.e., sample size), max_seq_len, vocab_size) so we'll need to be able to pad 3D sequences.

In [445]:
def pad_sequences(sequences, max_seq_len=0): # ACROSS DIFFERENT BATCHES WE CAN MAKE IT CONSISTENT, SAY max_seq_len=382
    """Pad sequences to max length in sequence."""
    max_seq_len = max(max_seq_len, max(len(sequence) for sequence in sequences))
    num_classes = sequences[0].shape[-1]
    padded_sequences = np.zeros((len(sequences), max_seq_len, num_classes))
    for i, sequence in enumerate(sequences):
        padded_sequences[i][:len(sequence)] = sequence
    return padded_sequences


In [446]:
# -- Test -- #
print (X[0].shape, X[1].shape, X[2].shape)
print(len(X[0]))
print(X[0])
padded = pad_sequences(X[0:3])
print (padded.shape) # (N (i.e., sample size), max_seq_len, vocab_size)
print (padded[0].shape, padded[1].shape, padded[2].shape)

(64, 47) (141, 47) (174, 47)
64
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(3, 174, 47)
(174, 47) (174, 47) (174, 47)


In [448]:
# -- Create padded dataset with consistent dimension --# 
padded_X =  pad_sequences(X[0:-1])
print(padded_X.shape)
print(len(X))
# print(X[10])

(824, 398, 47)
825


# CNN Model

In [449]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

In [477]:
class CNN(nn.Module):
    def __init__(self, vocab_size, num_filters, filter_size,
                 hidden_dim, dropout_p, num_classes):
        super(CNN, self).__init__()

        # Convolutional filters
        self.filter_size = filter_size # we'll used 1d filters like 1x3
        self.conv = nn.Conv1d(
            in_channels=vocab_size, out_channels=num_filters,
            kernel_size=filter_size, stride=1, padding=0, padding_mode="zeros") # padding 0 is no padding
        self.batch_norm = nn.BatchNorm1d(num_features=num_filters) 

        # FC layers
        self.fc1 = nn.Linear(num_filters, hidden_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.fc2 = nn.Linear(hidden_dim, num_classes)
        self.sigmoid = nn.Sigmoid()  # Sigmoid activation function for multi-label classification

    def forward(self, inputs, channel_first=False,):

        # Rearrange input so num_channels is in dim 1 (N, C, L)
        # With PyTorch, when dealing with convolution, our inputs (X) need to have the 
        # channels as the second dimension, so our inputs will be (N, vocab_size, max_seq_len)
#         x_in, = inputs
        x_in, = [i.squeeze(1) for i in inputs] # when transormation applied but not resizing
        if not channel_first:
            x_in = x_in.transpose(1, 2)

        # Padding for `SAME` padding
        max_seq_len = x_in.shape[2] # after transpose, the 3rd dim (i.e. 2) is the seq length
        padding_left = int((self.conv.stride[0]*(max_seq_len-1) - max_seq_len + self.filter_size)/2)
        padding_right = int(math.ceil((self.conv.stride[0]*(max_seq_len-1) - max_seq_len + self.filter_size)/2))

        # Conv outputs
        z = self.conv(F.pad(x_in, (padding_left, padding_right)))
        z = self.batch_norm(z)  # Batch normalization
        z = F.relu(z)  # Activation function
        z = F.max_pool1d(z, z.size(2)).squeeze(2) # a 1-dimensional max pooling operation to the input tensor z along its last dimension

        # FC layer
        z = self.fc1(z)
        z = self.dropout(z)
        z = self.fc2(z)
        z = self.sigmoid(z)  # Sigmoid activation function for multi-label classification
        return z


In [314]:
class CNN_2d(nn.Module):
    def __init__(self,num_filters, dropout_p, num_classes):
        super(CNN_2d, self).__init__()

        # Convolutional filters
#         self.filter_size = filter_size # we'll used 1d filters like 1x3
#         self.conv1 = nn.Conv2d(224,num_filters,3)
        self.conv1 = nn.Conv2d(1,num_filters,3)
        self.conv2 = nn.Conv2d(num_filters,100,3)
        self.conv3 = nn.Conv2d(100,num_filters,3)
        self.conv4 = nn.Conv2d(num_filters, 10, 3)
        
        # FC layers
        self.fc1 = nn.Linear(5760, 1024)
        self.dropout = nn.Dropout(dropout_p)
        self.fc2 = nn.Linear(1024, 512)
#         self.dropout = nn.Dropout(dropout_p)
        self.fc3 = nn.Linear (512, num_classes)
        self.sigmoid = nn.Sigmoid()
        
#         self.conv1 = nn.Conv1d(
#             in_channels=vocab_size, out_channels=num_filters,
#             kernel_size=filter_size, stride=1, padding=0, padding_mode="zeros") # padding 0 is no padding
        self.batch_norm1 = nn.BatchNorm2d(num_features=num_filters)
        self.batch_norm2 = nn.BatchNorm2d(num_features=100)

        # FC layers
  

    def forward(self, inputs, channel_first=False,):

        # Rearrange input so num_channels is in dim 1 (N, C, L)
        # With PyTorch, when dealing with convolution, our inputs (X) need to have the 
        # channels as the second dimension, so our inputs will be (N, vocab_size, max_seq_len)
        x_in, = inputs
#         if not channel_first:
#             x_in = x_in.transpose(1, 2)
#             x_in = x_in.transpose(0, 1)

        # Padding for `SAME` padding
#         max_seq_len = x_in.shape[2] # after transpose, the 3rd dim (i.e. 2) is the seq length
#         padding_left = int((self.conv1.stride[0]*(max_seq_len-1) - max_seq_len + 5)/2)
#         padding_right = int(math.ceil((self.conv1.stride[0]*(max_seq_len-1) - max_seq_len + 5)/2))

        # Conv outputs
#         z = self.conv1(F.pad(x_in, (padding_left, padding_right)))
        z = x_in
        z = F.relu(self.conv1(z))
        z = self.batch_norm1(z)  # Batch normalization
        z = F.max_pool2d(z, kernel_size = 2, stride = 2)
        z = F.relu(self.conv2(z))
        z = self.batch_norm2(z)  # Batch normalization
        z = F.max_pool2d(z, kernel_size = 2, stride = 2)
#         z = self.batch_norm(z)  # Batch normalization
        z = F.relu(self.conv3(z))
        z = self.batch_norm1(z)  # Batch normalization
        z = F.max_pool2d(z, kernel_size = 2, stride = 2)
        z = F.relu(self.conv4(z))
        
        z = torch.flatten(z, start_dim=1)
        
        z = F.relu(self.fc1(z))
        z = self.dropout(z)
        z = F.relu(self.fc2(z))
        z = self.dropout(z)
        z = self.fc3(z)
        z = self.sigmoid(z)
        return z


In [568]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.stride = stride

        # Adjust the dimensions if the number of input channels doesn't match the number of output channels
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(out_channels)
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        shortcut = self.shortcut(x)

        out += shortcut
        out = self.relu(out)
        return out

class ResNet1D(nn.Module):
    def __init__(self, block, layers, num_classes=23):
        super(ResNet1D, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv1d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self.make_layer(block, 64, layers[0])
        self.layer2 = self.make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self.make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self.make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Create a ResNet1D instance
model = ResNet1D(ResidualBlock, [2, 2, 2, 2], num_classes=23)
print(model)


ResNet1D(
  (conv1): Conv1d(1, 64, kernel_size=(7,), stride=(2,), padding=(3,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): ResidualBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(i

In [557]:
# Set device
cuda = True
device = torch.device("cuda" if (
    torch.cuda.is_available() and cuda) else "cpu")
torch.set_default_tensor_type("torch.FloatTensor")
if device.type == "cuda":
    torch.set_default_tensor_type("torch.cuda.FloatTensor")
print (device)

cpu


# Training

In [558]:
from torch.optim import Adam

In [559]:
## -- Trainer class to run single or multi-CV training (both exmaples are covered at the following)-- #
class Trainer(object):
    def __init__(self, model, device, loss_fn=None, optimizer=None, scheduler=None):

        # Set params
        self.model = model
        self.device = device
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.scheduler = scheduler

    def train_step(self, dataloader):
        """Train step."""
        # Set model to train mode
        self.model.train()
        loss = 0.0

        # Iterate over train batches
        for i, batch in enumerate(dataloader):

            # Step
#             batch = [item.to(self.device) for item in batch]  # Set device
            # Modify batch to move items to the device
#             batch = [[sub_item.squeeze(1).to(device) for sub_item in item] if isinstance(item, list) else item.squeeze(1).to(device) for item in batch]
            batch = [[sub_item.to(device) for sub_item in item] if isinstance(item, list) else item.to(device) for item in batch]
            inputs, targets = batch[:-1], batch[-1]
            self.optimizer.zero_grad()  # Reset gradients
            z = self.model(inputs)  # Forward pass
#             J = self.loss_fn(z, targets).item()  # Define loss
            J = self.loss_fn(z, targets.squeeze(1))  # Define loss
            J.backward()  # Backward pass
            self.optimizer.step()  # Update weights

            # Cumulative Metrics
            loss += (J.detach().item() - loss) / (i + 1)

        return loss

    def eval_step(self, dataloader):
        """Validation or test step."""
        # Set model to eval mode
        self.model.eval()
        loss = 0.0
        y_trues, y_probs = [], []

        # Iterate over val batches
        with torch.inference_mode():
            for i, batch in enumerate(dataloader):

                # Step
#                 batch = [item.to(self.device) for item in batch]  # Set device
                # Modify batch to move items to the device
#                 batch = [[sub_item.squeeze(1).to(device) for sub_item in item] if isinstance(item, list) else item.squeeze(1).to(device) for item in batch]
                batch = [[sub_item.to(device) for sub_item in item] if isinstance(item, list) else item.to(device) for item in batch]
                inputs, y_true = batch[:-1], batch[-1]

                z = self.model(inputs)  # Forward pass
#                 J = self.loss_fn(z, y_true).item()
                J = self.loss_fn(z, y_true.squeeze(1)).item()

                # Cumulative Metrics
                loss += (J - loss) / (i + 1)

                # Store outputs
#                 y_prob = F.softmax(z).cpu().numpy() # probably we don't need this as we already put sigmoid on the output layer
                y_prob = z.detach().numpy() # probably this is how we cast tensor to numpy
                y_probs.extend(y_prob)
                y_trues.extend(y_true.cpu().numpy())

        return loss, np.vstack(y_trues), np.vstack(y_probs)

    def predict_step(self, dataloader):
        """Prediction step."""
        # Set model to eval mode
        self.model.eval()
        y_probs = []

        # Iterate over val batches
        with torch.inference_mode():
            for i, batch in enumerate(dataloader):

                # Forward pass w/ inputs
                inputs, targets = batch[:-1], batch[-1]
                z = self.model(inputs)

                # Store outputs
                y_prob = F.softmax(z).cpu().numpy()
                y_probs.extend(y_prob)

        return np.vstack(y_probs)

    def train(self, num_epochs, patience, train_dataloader, val_dataloader):
        history = {'train_loss': [], 'test_loss': []}
        best_val_loss = np.inf
        for epoch in range(num_epochs):
            # Steps
            train_loss = self.train_step(dataloader=train_dataloader)
            val_loss, _, _ = self.eval_step(dataloader=val_dataloader)
            self.scheduler.step(val_loss)

            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model = self.model
                _patience = patience  # reset _patience
            else:
                _patience -= 1
            if not _patience:  # 0
                print("Stopping early!")
                break

            # Logging
            print(
                f"Epoch: {epoch+1} | "
                f"train_loss: {train_loss:.5f}, "
                f"val_loss: {val_loss:.5f}, "
                f"lr: {self.optimizer.param_groups[0]['lr']:.2E}, "
                f"_patience: {_patience}"
            )
            history['train_loss'].append(train_loss)
            history['test_loss'].append(val_loss)
        avg_train_loss = np.mean(history['train_loss'])
        avg_test_loss = np.mean(history['test_loss'])
#         print(avg_train_loss,avg_test_loss)
        return best_model, avg_train_loss, avg_test_loss

# Dataset processing

In [560]:
## this class includes sampling from different folds (its suitable for dataset creation when we're already
# applying transformation to the pre-padded dataset and passing it through dataloader ) ##
class Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y, max_filter_size, transform=None):
        self.X = X
        self.y = y
        self.max_filter_size = max_filter_size # this is padding arguement
#         self.sampler = sampler # sampling list
        self.transform = transform

    def __len__(self):
        return len(self.y)

    def __str__(self):
        return f"<Dataset(N={len(self)})>"

    def __getitem__(self, index):
        X = self.X[index]
        y = self.y[index]
        if self.transform:
            X = self.transform(X)
        return X, y

    def collate_fn(self, batch):
        """Custom collate function."""
        batch_X = [item[0] for item in batch]
        batch_y = [item[1] for item in batch]

        # Pad sequences (if necessary)
#         batch_X = pad_sequences(batch_X, max_seq_len=self.max_filter_size) # padding for consistent dimensions for each input batch


        # Convert batch_y to tensors and reshape
#         batch_y = torch.stack([torch.LongTensor(y).view(1, -1) for y in batch_y], dim=0)
#         batch_y = torch.stack([torch.FloatTensor(y).view(1, -1) for y in batch_y], dim=0)
        batch_y = torch.stack([torch.tensor(y, dtype=torch.float32).unsqueeze(0) for y in batch_y], dim=0)


        # Cast
#         batch_X = torch.FloatTensor(batch_X) # float because input is kind of continuous in the pixel space
        batch_X = torch.stack(batch_X)

        return batch_X, batch_y


    def create_dataloader(self, batch_size, sampler=None, shuffle=False, drop_last=True):
        return torch.utils.data.DataLoader(
            dataset=self, batch_size=batch_size, sampler=sampler, collate_fn=self.collate_fn,
            shuffle=shuffle, drop_last=drop_last, pin_memory=True) 


# K-fold CV - data augmentation

In [561]:
from sklearn.model_selection import KFold, StratifiedKFold
import random
from torch.utils.data import random_split,SubsetRandomSampler, ConcatDataset

In [562]:
k=5
splits=KFold(n_splits=k,shuffle=True,random_state=42)
# splits=StratifiedKFold(n_splits=k,shuffle=True,random_state=42)

In [563]:
import torchvision.transforms as transforms
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [564]:
c = 0
for i, batch in enumerate(train_dataloader):
    c = c+1
    print("count: ", c)
#     batch = [[sub_item.squeeze(1).to(device) for sub_item in item] if isinstance(item, list) else item.squeeze(1).to(device) for item in batch]
    batch = [[sub_item.to(device) for sub_item in item] if isinstance(item, list) else item.to(device) for item in batch]
    inputs, y_true = batch[:-1], batch[-1]
#     x_in, = inputs
    x_in, = [i.squeeze(1) for i in inputs]
    x_in = x_in.transpose(1, 2)
#     z = model(x_in)  # Forward pass
#                 J = self.loss_fn(z, y_true).item()
#     J = self.loss_fn(z, y_true.squeeze(1)).item()
    print(len(batch))
#     print(batch[0].shape, batch[1].shape)
    print(len(inputs), inputs[0].shape)
    print(len(y_true), y_true[0].shape)
    print(x_in.shape)
#     print(z.shape)
    print(y_true.shape)


# for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(X)))):
#     train_data_X = []
#     train_data_y = []
#     test_data_X = []
#     test_data_y = []

#     print('Fold {}'.format(fold + 1))
    
#     for i in train_idx:  
#         train_data_X.append(padded_X[i-1].astype(np.uint8))
#         train_data_y.append(y[i-1,:].astype(np.uint8))
    
#     for j in val_idx:
#         test_data_X.append(padded_X[j-1].astype(np.uint8))
#         test_data_y.append(y[j-1,:].astype(np.uint8))
#     train_dataset = Dataset(X=train_data_X, y=train_data_y, max_filter_size=400, transform=train_transforms) 
#     test_dataset = Dataset(X=test_data_X, y=test_data_y, max_filter_size=400, transform=train_transforms) 
#     print(len(train_data_X), len(train_data_y))
#     print(len(test_data_X), len(test_data_y))
#     print(train_dataset[0][0].shape, test_dataset[0][0].shape)
#     print(train_dataset[0][1].shape, test_dataset[0][1].shape)

count:  1
2
1 torch.Size([32, 1, 224, 224])
32 torch.Size([1, 23])
torch.Size([32, 224, 224])
torch.Size([32, 1, 23])
count:  2
2
1 torch.Size([32, 1, 224, 224])
32 torch.Size([1, 23])
torch.Size([32, 224, 224])
torch.Size([32, 1, 23])
count:  3
2
1 torch.Size([32, 1, 224, 224])
32 torch.Size([1, 23])
torch.Size([32, 224, 224])
torch.Size([32, 1, 23])
count:  4
2
1 torch.Size([32, 1, 224, 224])
32 torch.Size([1, 23])
torch.Size([32, 224, 224])
torch.Size([32, 1, 23])
count:  5
2
1 torch.Size([32, 1, 224, 224])
32 torch.Size([1, 23])
torch.Size([32, 224, 224])
torch.Size([32, 1, 23])
count:  6
2
1 torch.Size([32, 1, 224, 224])
32 torch.Size([1, 23])
torch.Size([32, 224, 224])
torch.Size([32, 1, 23])
count:  7
2
1 torch.Size([32, 1, 224, 224])
32 torch.Size([1, 23])
torch.Size([32, 224, 224])
torch.Size([32, 1, 23])
count:  8
2
1 torch.Size([32, 1, 224, 224])
32 torch.Size([1, 23])
torch.Size([32, 224, 224])
torch.Size([32, 1, 23])
count:  9
2
1 torch.Size([32, 1, 224, 224])
32 torch.Siz

In [517]:
# print(len(X), y.shape)
# print(X[0])
# print(X[i].astype(np.int64))

In [565]:
# This is necessary as our dataset is massively skewed to positives for all classes
def calculate_pos_weights(data): # data = labels
#     class_vectors = np.array(data)
    class_counts = np.sum(data, axis=0)
#     pos_weights = np.ones_like(class_counts)
    neg_counts = [len(data)-pos_count for pos_count in class_counts]
    pos_weights = []
    for i in range(len(class_counts)):
        pos_weights.append(neg_counts[i]/ class_counts[i] + 1e-5)
    return torch.as_tensor(np.array(pos_weights), dtype=torch.float).to(device)

In [566]:
NUM_FILTERS = 50
HIDDEN_DIM = 300
DROPOUT_P = 0.2
FILTER_SIZE = 3
# NUM_CLASSES = test_dataset[0][1].shape[0]
NUM_CLASSES = y.shape[1]
print(NUM_CLASSES)

23


In [573]:
# Initialize model
# model = CNN(vocab_size=224, num_filters=NUM_FILTERS, filter_size=FILTER_SIZE,
#             hidden_dim=HIDDEN_DIM, dropout_p=DROPOUT_P, num_classes=NUM_CLASSES)
# model = CNN(vocab_size=224, num_filters=NUM_FILTERS, filter_size=FILTER_SIZE,
#             hidden_dim=HIDDEN_DIM, dropout_p=DROPOUT_P, num_classes=NUM_CLASSES)
# model = CNN_2d(num_filters=NUM_FILTERS, dropout_p=DROPOUT_P, num_classes=NUM_CLASSES)
# model = model.to(device) # set device
# print (model.named_parameters)

In [569]:
LEARNING_RATE = 1e-3
PATIENCE = 10
NUM_EPOCHS = 30

# --define class weights
class_weights = calculate_pos_weights(data=y) # positive weights
print(class_weights)

#--Define Loss
loss_fn = nn.BCEWithLogitsLoss(pos_weight=class_weights)

#--Define optimizer & scheduler
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.1, patience=3)

tensor([0.0550, 0.0591, 0.0827, 0.2009, 0.1587, 0.0300, 0.0061, 0.0073, 0.1411,
        0.0160, 0.0123, 0.0037, 0.0813, 0.0274, 0.1702, 0.0135, 0.0404, 0.0645,
        0.0870, 0.0673, 0.0274, 0.0160, 0.0061])


In [570]:
# X: [16, 49, 38]
#  y: [16, 1, 23]
# print(len(X))

In [574]:
import math
from sklearn.metrics import multilabel_confusion_matrix, classification_report
from sklearn.metrics import hamming_loss, accuracy_score, precision_recall_fscore_support
from sklearn.metrics import roc_auc_score

# Create dataloaders
batch_size = 32
k=5
splits=KFold(n_splits=k,shuffle=True)
# splits=StratifiedKFold(n_splits=k,shuffle=True) # Supported target types are: ('binary', 'multiclass'). Got 'multilabel-indicator' instead.. so we cannot use it


# Trainer module
trainer = Trainer(
    model=model, device=device, loss_fn=loss_fn,
    optimizer=optimizer, scheduler=scheduler)

    
history = {'Hamming loss': [], 'miP': [], 'miR': [], 'miF1': [], 'maP': [], 'maR': [], 'maF1': [], 'micro_auc': [], 'macro_auc': [] }
for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(X)))):
    train_data_X = []
    train_data_y = []
    test_data_X = []
    test_data_y = []

    print('Fold {}'.format(fold + 1))
    
    for i in train_idx:  
#         train_data_X.append(X[i].astype(np.uint8))
#         train_data_y.append(y[i,:].astype(np.uint8))
     # -- sampling from already padded dataframe --#
        train_data_X.append(padded_X[i-1].astype(np.uint8))
        train_data_y.append(y[i-1,:].astype(np.uint8))
    
    for j in val_idx:
#         test_data_X.append(X[j].astype(np.uint8))
#         test_data_y.append(y[j,:].astype(np.uint8))
     # -- sampling from already padded dataframe --#
        test_data_X.append(padded_X[j-1].astype(np.uint8))
        test_data_y.append(y[j-1,:].astype(np.uint8))
    
    train_dataset = Dataset(X=train_data_X, y=train_data_y, max_filter_size=400, transform=train_transforms) 
    test_dataset = Dataset(X=test_data_X, y=test_data_y, max_filter_size=400, transform=test_transforms)
#     print(len(train_dataset))
#     print(len(test_dataset))
    
    train_dataloader = train_dataset.create_dataloader(batch_size=batch_size)
    val_dataloader = test_dataset.create_dataloader(batch_size=batch_size)
    batch_X, batch_y = next(iter(val_dataloader))
    print ("Sample batch:\n"
    f"  X: {list(batch_X.size())}\n"
    f"  y: {list(batch_y.size())}\n"
    f"  X_shape: {batch_X[0].shape, batch_X[10].shape}\n"
    f"  y_shape: {batch_y[0].shape, batch_y[10].shape}\n")
    
    best_model, avg_train_loss,avg_test_loss = trainer.train(
        NUM_EPOCHS, PATIENCE, train_dataloader, val_dataloader)
    
    print(f'Performance of {fold} fold:')
    print("Average Training Loss: {:.4f} \t Average Test Loss: {:.4f} ".format(avg_train_loss,avg_test_loss)) 
    
    # For evaluation
    test_loss, y_true, y_pred_prob = trainer.eval_step(dataloader=val_dataloader)

    # 2. Apply thresholding to convert probabilities to binary predictions
    threshold = 0.5
    y_pred_binary = (y_pred_prob > threshold).astype(float)

    # 3. Calculate evaluation metrics
    hamming_loss_value = hamming_loss(y_true, y_pred_binary)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred_binary, average='micro')
    ma_precision, ma_recall, ma_f1, _ = precision_recall_fscore_support(y_true, y_pred_binary, average='macro')
    
    # Macro-average AUC
    if np.sum(np.all(y_true == 1, axis=0)) > 0: ## this is needed because macro auc cannot be calculated if one label of all samples belong to either +ve/-ve
        # Find cols where every element is 1
        columns_to_remove = np.all(y_true == 1, axis=0)

        # Remove those rows
        y_true_filtered = y_true[:, ~columns_to_remove]
        y_pred_filtered = y_pred_prob[:, ~columns_to_remove]
        macro_auc = roc_auc_score(y_true_filtered, y_pred_filtered, average='macro')
        print(f'done maauc but with {np.sum(np.all(y_true == 1, axis=0))} labels removed')
    else:
        macro_auc = roc_auc_score(y_true, y_pred_prob, average='macro')
        print('done maauc') 
    
    # Micro-auc
    try:
        micro_auc = roc_auc_score(y_true.ravel(), y_pred_prob.ravel(), average='micro', multi_class='ovr')
        print('done miauc')
    except ValueError:
        pass
#     micro_auc = roc_auc_score(y_true, y_pred_prob)

#     # Macro-average AUC
#     macro_auc = roc_auc_score(y_true, y_pred_prob, average='macro')
#     print('done maauc')

    history['Hamming loss'].append(hamming_loss_value)
    history['miP'].append(precision)
    history['miR'].append(recall)
    history['miF1'].append(f1)
    history['maP'].append(ma_precision)
    history['maR'].append(ma_recall)
    history['maF1'].append(ma_f1)
    history['micro_auc'].append(micro_auc)
    history['macro_auc'].append(macro_auc)
    # Print or use the evaluation metrics as needed
    print("Hamming Loss: {:.4f} \t miP: {:.4f} \t miR: {:.4f} \t miF1: {:.4f} \t maP: {:.4f} \t maR: {:.4f} \t maF1: {:.4f} \t micro_auc: {:.4f} \t macros_auc: {:.4f} ".format(hamming_loss_value, precision, recall, f1, ma_precision, ma_recall, ma_f1, micro_auc, macro_auc))
    

# print(train_dataset[742])
# print(train_dataset[742][0].shape)

Fold 1
Sample batch:
  X: [32, 1, 224, 224]
  y: [32, 1, 23]
  X_shape: (torch.Size([1, 224, 224]), torch.Size([1, 224, 224]))
  y_shape: (torch.Size([1, 23]), torch.Size([1, 23]))



TypeError: conv1d() received an invalid combination of arguments - got (list, Parameter, NoneType, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!list!, !Parameter!, !NoneType!, !tuple!, !tuple!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!list!, !Parameter!, !NoneType!, !tuple!, !tuple!, !tuple!, int)


In [870]:
# print(y_true)

In [545]:
print("Average performance over all folds: ".format(avg_train_loss,avg_test_loss))

print("Hamming Loss:", np.mean(history['Hamming loss']))
print("miP:", np.mean(history['miP']))
print("miR:", np.mean(history['miR']))
print("miF1:", np.mean(history['miF1']))
print("maP:", np.mean(history['maP']))
print("maR:", np.mean(history['maR']))
print("maF1:", np.mean(history['maF1']))
print("micro_auc:", np.mean(history['micro_auc']))
print("macro_auc:", np.mean(history['macro_auc']))

Average performance over all folds: 
Hamming Loss: 0.6928260869565217
miP: 0.9868105415691197
miR: 0.2722008419312977
miF1: 0.42624270710277246
maP: 0.8014362388312268
maR: 0.2650477337049506
maF1: 0.3635958786167487
micro_auc: 0.7217419779929172
macro_auc: 0.7183479796382202


In [546]:
# for i in range(y_true.shape[0]):
#     print(y_true[i], y_pred_binary[i])
#     print(y_pred_prob[i])
# test_loss, y_true, y_pred_prob = trainer.eval_step(dataloader=val_dataloader)
# print(y_true[:9])
# print(y_pred_prob[:9])
# print(np.sum(y_pred_prob, axis=1))

To do:

1. run a better 1d Conv model with the same data aug
2. run a 1d resnet
2. run without resizing